# 3th_homework

### Import Section

In [4]:
import numpy as np
import pandas as pd

import implicit

### Global Settings Section

In [5]:
# random_state_global = 0

### Path Section

In [6]:
# PATH_DATA_

## Theoretical Part

1. <Вопрос 1>

    User-User рекомендации строятся на оценке эмбеддинга пользователя:
        - эмбеддинг пользователя;
        - поиск наиболее похожих пользователей;
        - подготовка рекомендаций на основе усреднённой оценки пользователей.
    
    Item-Item рекомендации, аналогично, используют эмбеддинг продукта:
        - эмбеддинг продукта;
        - поиск наиболее похожих продуктов;
        - рекомендация новых товаров для пользователя.
    
    Схожесть заключается в эмбеддинге и поиске похожих пользователей или продуктов. Различия - для продуктов достаточно рекомендовать наиболее похожие продукты согласно некой мере схожести, для пользователей нужна некая грруповая оценка новых продуктов.
    
    Поэтому без доработок функцию вряд ли получится применить.
    
    ALS отличается от Item-Item тем, что использует категоризацию пользователей и продуктов. Item-Item использует наиболее похожие продукты, а ALS наиболее близкие категории как для продуктов, так и для пользователей. Затем для пользователей с соответствующей категорией могут быть подобраны наиболее вероятные продукты из этой категории.

**Заглушка, до 04.04.22 постараюсь выполнить работу.**